In [4]:
import PIL.Image as Image
import os
import cv2
import numpy as np
import pandas as pd

import matplotlib.pylab as plt
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers,losses
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

In [5]:
dataset_path = 'Normal Gait'
dataset_path = 'abnormal'

In [6]:
# Creating dataframe by iterating through filepaths
import pathlib 
abnormal_dir= pathlib.Path("abnormal/")
healthy_dir=pathlib.Path("Normal Gait/")

In [7]:
dirlist=[abnormal_dir,healthy_dir ]
classes=['Yes', 'No']
filepaths=[]
labels=[]
for i,j in zip(dirlist, classes):
    filelist=os.listdir(i)
    for f in filelist:
        filepath=os.path.join (i,f)
        filepaths.append(filepath)
        labels.append(j)
print ('filepaths: ', len(filepaths), '   labels: ', len(labels))

filepaths:  23560    labels:  23560


In [8]:
images_dict = {
    'Cancer': list(abnormal_dir.glob('*')) ,   
    'healthy': list(healthy_dir.glob('*'))  
}

labels_dict = {
    'Cancer': "cancer",
    'healthy': "healthy",
    
}

In [9]:
img= cv2.imread(str(images_dict['healthy'][0]))
img.shape

(240, 320, 3)

In [10]:
X, y = [], []
for name,images in images_dict.items():#returns key , path for each record
    for image in images:
        img=str(image)
     
        #resize_img=cv2.resize(img,(224,224))
       
        X.append(img)
        y.append(labels_dict[name])

In [11]:
Fseries = pd.Series(X, name="filepaths",dtype=str)
Lseries = pd.Series(y, name="labels",dtype=str)
tumor_data = pd.concat([Fseries,Lseries], axis=1)
tumor_df = pd.DataFrame(tumor_data)

print(tumor_df.head())
print("---------------------")
print(tumor_df["labels"].value_counts())

              filepaths  labels
0      abnormal\006.png  cancer
1  abnormal\007 (2).png  cancer
2  abnormal\007 (3).png  cancer
3  abnormal\007 (4).png  cancer
4  abnormal\007 (5).png  cancer
---------------------
labels
cancer     14043
healthy     9517
Name: count, dtype: int64


In [12]:
tumor_df.shape

(23560, 2)

In [13]:
#splitting data
from sklearn.model_selection import train_test_split
train_images, test_images = train_test_split(tumor_df, test_size=0.2, random_state=32)
train_set, val_set = train_test_split(tumor_df, test_size=0.2, random_state=32)

In [14]:
#shape of splitted data
print(train_set.shape)
print(test_images.shape)
print(val_set.shape)
print(train_images.shape)

(18848, 2)
(4712, 2)
(4712, 2)
(18848, 2)


In [15]:
#Generate batches of tensor image data with real-time data augmentation.
image_gen = ImageDataGenerator(preprocessing_function= tf.keras.applications.mobilenet_v2.preprocess_input)
train = image_gen.flow_from_dataframe(dataframe= train_set,x_col="filepaths",y_col="labels",
                                      target_size=(255,255),
                                      color_mode='rgb',
                                      class_mode="categorical", #used for Sequential Model
                                      batch_size=32,
                                      shuffle=False            #do not shuffle data
                                     )
test = image_gen.flow_from_dataframe(dataframe= test_images,x_col="filepaths", y_col="labels",
                                     target_size=(255,255),
                                     color_mode='rgb',
                                     class_mode="categorical",
                                     batch_size=32,
                                     shuffle= False
                                    )
val = image_gen.flow_from_dataframe(dataframe= val_set,x_col="filepaths", y_col="labels",
                                    target_size=(244,244),
                                    color_mode= 'rgb',
                                    class_mode="categorical",
                                    batch_size=32,
                                    shuffle=False
                                   )

Found 18848 validated image filenames belonging to 2 classes.
Found 4712 validated image filenames belonging to 2 classes.
Found 4712 validated image filenames belonging to 2 classes.


In [16]:
classes=list(train.class_indices.keys())
print (classes)

['cancer', 'healthy']


In [24]:
#Defining Sequential Model
cnn= Sequential([
                        #cnn layer 
                        layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding='same',input_shape=(255,255,3)),
                        layers.MaxPool2D(pool_size=(2,2)),

                        layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding='same'),
                        layers.MaxPool2D(pool_size=(2,2)),

                        layers.Flatten(),
                        layers.Dense(64, activation='relu'),
                        layers.Dropout(rate=0.3),
                        layers.Dense(32, activation="relu"),
                        layers.Dense(2, activation="sigmoid")

])

In [25]:
cnn.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [26]:
cnn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 255, 255, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 127, 127, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 127, 127, 32)      9248      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 127008)            0         
                                                                 
 dense_9 (Dense)             (None, 64)               

In [27]:
History = cnn.fit(train, validation_data= val, epochs=5,verbose=1)

Epoch 1/5
589/589 [==============================] - ETA: 0s - loss: 0.4793 - accuracy: 0.7966

InvalidArgumentError: Graph execution error:

Detected at node 'sequential_3/dense_9/Relu' defined at (most recent call last):
    File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Gait Analysis\venv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Gait Analysis\venv\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "c:\Gait Analysis\venv\lib\site-packages\ipykernel\kernelapp.py", line 728, in start
      self.io_loop.start()
    File "c:\Gait Analysis\venv\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_l